In [ ]:
import Pkg

pkgs = [
"ProgressMeter",
"uCSV",
"DataFrames"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

In [ ]:
base_directory = dirname(pwd())
metadata_file_path = joinpath(base_directory, "metadata", "ultima", "Snyder_delivery_1.xlsx - Sheet1.tsv")
metadata_table = DataFrames.DataFrame(uCSV.read(metadata_file_path, delim='\t', header=1))
sort!(metadata_table, "Customer ID")
ultima_data_dir = joinpath(base_directory, "data", "Ultima")

In [ ]:
for sample_metadata in DataFrames.eachrow(metadata_table)
    sample_directory = mkpath(joinpath(ultima_data_dir, sample_metadata["Customer ID"]))
    for remote_file in [
        sample_metadata["raw fastq"],
        sample_metadata["non-human fastq"],
        sample_metadata["kraken report all data, incl human"]
    ]
        local_file = joinpath(sample_directory, basename(remote_file))
        if !isfile(local_file)
            cmd = `aws s3 cp $(remote_file) $(local_file)`
            run(cmd)
        else
            println("$(remote_file) already downloaded...")
        end
    end
end

In [ ]:
# samples are unique by ID & pool, but same IDs are on multiple pools
# unique(metadata_table[!, ["pool nexus", "Customer ID"]])

In [ ]:
# conda create -n awscli -c conda-forge awscli
# pip3 install awscli --upgrade --user